In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import os
from tqdm import tqdm as tqdm
import datetime

In [2]:
page_path_whole = '/home3/r05322021/Desktop/FB Data/Polarization/page_score/whole/'
page_score_file_whole = os.listdir(page_path_whole)
page_path_all = '/home3/r05322021/Desktop/FB Data/original_ideology/page_score/2015-05-03_to_2016-11-26_4weeks/'
page_score_file_all = os.listdir(page_path_all)
page_path_new = '/home3/r05322021/Desktop/FB Data/Polarization/page_score/new/all/'
page_score_file_new = os.listdir(page_path_new)
page_path_quit = '/home3/r05322021/Desktop/FB Data/Polarization/page_score/quit/all/'
page_score_file_quit = os.listdir(page_path_quit)

In [3]:
df_whole_list, df_all_list, df_new_list, df_quit_list = [], [], [], []
for f in page_score_file_whole:
    df = pd.read_csv(f'{page_path_whole}{f}', converters={'page_id': str}, engine='python')
    df['Week'] = f[-14:-4]
    df_whole_list.append(df)
for f in page_score_file_all:
    df = pd.read_csv(f'{page_path_all}{f}', converters={'page_id': str}, engine='python')
    df['Week'] = f[-14:-4]
    df_all_list.append(df)
for f in page_score_file_new:
    df = pd.read_csv(f'{page_path_new}{f}', converters={'page_id': str}, engine='python')
    df['Week'] = f[-14:-4]
    df_new_list.append(df)
for f in page_score_file_quit:
    df = pd.read_csv(f'{page_path_quit}{f}', converters={'page_id': str}, engine='python')
    df['Week'] = f[-14:-4]
    df_quit_list.append(df)
df_whole = pd.concat(df_whole_list, axis=0)
df_all = pd.concat(df_all_list, axis=0)
df_new = pd.concat(df_new_list, axis=0)
df_quit = pd.concat(df_quit_list, axis=0)

In [4]:
std_whole = pd.DataFrame([[d['Week'].iloc[0], d['PC1_std'].std()] for d in [group[1] for group in df_whole.groupby('Week')]], columns=['Week', 'std_whole'])
std_all = pd.DataFrame([[d['Week'].iloc[0], d['PC1_std'].std()] for d in [group[1] for group in df_all.groupby('Week')]], columns=['Week', 'std_all'])
std_new = pd.DataFrame([[d['Week'].iloc[0], d['PC1_std'].std()] for d in [group[1] for group in df_new.groupby('Week')]], columns=['Week', 'std_new'])
std_quit = pd.DataFrame([[d['Week'].iloc[0], d['PC1_std'].std()] for d in [group[1] for group in df_quit.groupby('Week')]], columns=['Week', 'std_quit'])

In [39]:
std_whole['Week'] = std_whole['Week'].apply(lambda x: str(datetime.datetime.strptime(x, '%Y-%m-%d').date() - datetime.timedelta(days=6)))
std_all['Week'] = std_all['Week'].apply(lambda x: str(datetime.datetime.strptime(x, '%Y-%m-%d').date() - datetime.timedelta(days=6)))
std_new['Week'] = std_new['Week'].apply(lambda x: str(datetime.datetime.strptime(x, '%Y-%m-%d').date() - datetime.timedelta(days=6)))
std_quit['Week'] = std_quit['Week'].apply(lambda x: str(datetime.datetime.strptime(x, '%Y-%m-%d').date() - datetime.timedelta(days=6)))

In [13]:
reaction = pd.read_csv('/home3/r05322021/Desktop/FB Data/issue_reaction/whole/reaction_weekly.csv')
user = pd.read_csv('/home3/r05322021/Desktop/FB Data/issue_reaction/whole/user_weekly.csv')
post = pd.read_csv('/home3/r05322021/Desktop/FB Data/issue_reaction/whole/post_weekly.csv')
reaction = reaction[reaction.Week < '2016-11-27']
user = user[user.Week < '2016-11-27']
post = post[post.Week < '2016-11-27']

In [15]:
reaction_immigration = reaction[reaction.type == 'immigration'][['Week', 'amount', 'ratio']].rename(columns={'amount': 'immigration_reaction_amount', 'ratio': 'immigration_reaction_ratio'})
reaction_race = reaction[reaction.type == 'race'][['Week', 'amount', 'ratio']].rename(columns={'amount': 'race_reaction_amount', 'ratio': 'race_reaction_ratio'})
user_immigration = user[user.type == 'immigration'][['Week', 'amount', 'ratio']].rename(columns={'amount': 'immigration_user_amount', 'ratio': 'immigration_user_ratio'})
user_race = user[user.type == 'race'][['Week', 'amount', 'ratio']].rename(columns={'amount': 'race_user_amount', 'ratio': 'race_user_ratio'})
post_immigration = post[post.type == 'immigration'][['Week', 'amount', 'ratio']].rename(columns={'amount': 'immigration_post_amount', 'ratio': 'immigration_post_ratio'})
post_race = post[post.type == 'race'][['Week', 'amount', 'ratio']].rename(columns={'amount': 'race_post_amount', 'ratio': 'race_post_ratio'})

In [27]:
FB_df = pd.merge(reaction_immigration, reaction_race, how='inner', on=['Week'])
for d in [user_immigration, user_race]:
    FB_df = pd.merge(FB_df, d, how='inner', on=['Week'])
for d in [post_immigration, post_race]:
    FB_df = pd.merge(FB_df, d, how='inner', on=['Week'])
for col in ['reaction_amount', 'reaction_ratio', 'user_amount', 'user_ratio', 'post_amount', 'post_ratio']:
    FB_df[f'related_{col}'] = FB_df[f'immigration_{col}'] + FB_df[f'race_{col}']

In [42]:
for df in [std_whole, std_all, std_new, std_quit]:
    FB_df = pd.merge(FB_df, df, how='inner', on='Week')

In [44]:
FB_df.to_csv('/home3/r05322021/Desktop/FB Data/Polarization/polarization_issue.csv', index=False)